# MCP Examples

MCP server resources:
- https://github.com/modelcontextprotocol/servers?tab=readme-ov-file
- https://mcp.so
- https://glama.ai/mcp
- https://smithery.ai/
- https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries

There are two types of MCP servers:
- Running on a local machine
- Running on a remote server

In [ ]:
# improts and env setup
from agents.mcp import MCPServerStdio
from agents import Agent, Runner, trace
from dotenv import load_dotenv
load_dotenv(override=True)

### Running an MCP server in local machine with openai agents sdk

We run an MCP server localy by using `MCPServerStdio` from openai-agents SDK. These servers may be launched using `uvx`, `node`, or `docker`, depending on the server implementation

In [ ]:
# Let's see the list of tools provided by `fetch` from Anthropic
fetch_parameters = {"command": "uvx", "args": ["mcp-server-fetch"]}
async with MCPServerStdio(params=fetch_parameters, client_session_timeout_seconds=60) as fetch_server:
    fetch_tools_mcp = await fetch_server.list_tools()
    print("Fetch tools:", fetch_tools_mcp)


### Using tools from MCP by an agent

Doing this example may take a while to run.

In [ ]:
# Run MCP Server STDIO
internet_search_agent_instructions = """
    You are an internet search agent that can fetch and summarize web content.
    When given a URL or asked to search for information, use the fetch tool to retrieve web pages.
    Always provide a clear, concise summary of the content you find.
    If a page is too long, focus on the most relevant information for the user's query.
"""
fetch_parameters = {"command": "uvx", "args": ["mcp-server-fetch"]}
async with MCPServerStdio(params=fetch_parameters, client_session_timeout_seconds=60) as fetch_server:
    fetch_tools_mcp = await fetch_server.list_tools()
    agent = Agent(
        name="Internet search agent",
        instructions=internet_search_agent_instructions,
        model="gpt-4o-mini",
        mcp_servers=[fetch_server],
    )
    with trace("search using fetch tools from MCP"):
        result = await Runner.run(agent, "Find information on how to lose weight fast")
        print(result.final_output)